# SSScoring interactive quickstart

SSScoring provides analsysis tools for individual or bulk processing of FlySight GPS competition data gathered during speed skydiving training and competition.  Scoring methodology adheres to International Skydiving Commission (ISC), International Speed Skydiving Association (ISSA), and United States Parachute Association (USPA) published competition and scoring rules.  SSScoring libraries and tools also operate with track data files produced by these devices:

- FlySight 1
- FlySight 2
- SkyTrax GPS and barometric device

SSScoring leverages data manipulation tools in the pandas and NumPy data analysis libraries.  All the SSScoring code is written in pure Python, but the implementation leverages libraries that may require native code for GPU and AI chipset support like Nvidia and M-chipsets.


---
## Requirements

- A data lake containing track data files
- Some prior Python programming to make changes to this live document or to write new implementations
- Knowledge of pandas and NumPy is a nice-to-have, but not a show-stopper

To start let's create a data lake and copy one or more of these:

- FlySight version 1 files in the form `HH-mm-ss.CSV`
- FlySight version 2 data container in the form `YY-MM-dd/HH-mm-ss/*CSV`

The `./quickstart` directory will be our data lake.  It could contain zero files or everything on a whole disk.

Check if this prerequisite is fulfilled:

In [ ]:
!mkdir -p "./quickstart-example" && if [[ -d "./quickstart-example" ]]; then ls -Al "./quickstart-example"; fi

It doesn't matter where the data lake is located.  It could be on the local device, the FlySight device itself (though reading files from it is cumbersome and super slow), a cloud drive, Amazon S3, anywhere that the system is accessible.  The quickstart data lake already contains a mix of FlySight 1 and 2 files.  Feel free to replace them with your own files.

---
## Use case:  scoring all the jumps of a competition in 12 lines of code or less

As a speed skydiving judge, I have a directory containing a competitor's FlySight files.  I want to get the score for each jump, the aggregate score, the mean speed, and the maximum speed.  It was a short competition consisting of only 3 jumps.

### Imports constants, and getting all tracks

The previous command shows that our data lake contains files of various types, including CSVs.  SSScoring can scan a directory and its subdirectories and returns a list of all valid speed skydiving files.

In [ ]:
from ssscoring.flysight import getAllSpeedJumpFilesFrom

import pathlib

DATA_LAKE = pathlib.Path('quickstart-example/')

allFiles = getAllSpeedJumpFilesFrom(DATA_LAKE)

print('\n'.join((f.as_posix() for f in allFiles.keys())))

`allFiles` is an ordered dictionary in which each entry has these attributes set:

- a `libpath.Path` object associated with the track file
- a FlySight version tag set to 1 or 2

In [ ]:
display(allFiles)

`allFiles` includes what looks like a valid FlySight file - the one with the lower case `.csv` extension.  Compatible third-party devices like SkyTrax have a different output format and use a lower case file extension.  That's okay.  SSScoring will process their output as long as the track has the correct data headers and the file extension is CSV or csv.

### Get all the results

This is super easy--barely an inconvenience.  SSScoring sets AGL == MSL if the caller omits the optional `altitudeDZMeters` argument:

In [ ]:
from ssscoring.calc import processAllJumpFiles

jumpResults = processAllJumpFiles(allFiles, altitudeDZMeters = 616)

The results are stored in a dictionary that uses a tagged track name as the key, and a **<a href='https://pr3d4t0r.github.io/SSScoring/ssscoring/datatypes.html#JumpResults' target='_blank'>JumpResult object</a>**:

Results keys:

In [ ]:
print('\n'.join((r for r in jumpResults.keys())))

Viewing a specific result can be overwhelming.  This use case is about reporting the results of the competition with as little fuss and as fast as possible:

In [ ]:
from ssscoring.calc import aggregateResults
from ssscoring.calc import roundedAggregateResults

summary = aggregateResults(jumpResults)
summary

Results can also be rounded, for convenience:

In [ ]:
roundedAggregateResults(summary)

And of course, we can resolve final results from the summary table:

In [ ]:
print('Total score = %5.2f, mean speed = %5.2f' % (summary.score.sum(), summary.score.mean()))

### Putting it all together

There was a lot of information to unpack for this use case because the software is doing a lot of work.  In everyday use, however, usage is quite simple.  A complete scoring system can be developed in 10 lines of code:

In [ ]:
from ssscoring.calc import aggregateResults
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import roundedAggregateResults
from ssscoring.flysight import getAllSpeedJumpFilesFrom

import pathlib

DATA_LAKE = pathlib.Path('quickstart-example/')
allFiles = getAllSpeedJumpFilesFrom(DATA_LAKE)
jumpResults = processAllJumpFiles(allFiles, altitudeDZMeters = 616)
display(aggregateResults(jumpResults))
print('Total score = %5.2f, mean speed = %5.2f' % (summary.score.sum(), summary.score.mean()))

If we wanted to get technical the whole implementation can be done in fewer than 5 lines of code.  The import statements do nothing other than make the functions available for processing.  Actual coding starts with the data lake definition.

---
## Use case:  analyzing and scoring a single jump